In [67]:
import requests as r
import json
import pyttsx3
from moviepy.editor import *

In [68]:
YOUTUBE_API_KEY = ""
VOICE_ENGINE = pyttsx3.init()
MIN_UPVOTES = 100
MIN_CHARACTERS = 1000
IS_TEXT = True
NUM_SUBCLIPS = 7
NUM_SHORTS_FOR_LONG_FORM = ...

In [69]:

def get_posts():
    with open("subreddits.txt", "r") as file:
        for sub in file.readlines():
            posts = r.get(f"https://www.reddit.com/r/{sub}/hot/.json", headers={"User-agent": "tester 0.1"}).content
            posts = json.loads(posts)["data"]["children"]
            posts = [i for i in posts
                     if len(i["data"]["selftext"]) >= MIN_CHARACTERS
                     and i["data"]["score"] >= MIN_UPVOTES
                     and i["data"]["media"] is None
                     and i["data"]["is_video"] is not IS_TEXT]
    return posts


def get_tts(posts):
    for i in posts:
        title = i["data"]["title"].replace("/", "slash").replace("\\", "slash")
        text = f"r/{i['data']['subreddit']}. {i['data']['selftext']}"
        VOICE_ENGINE.save_to_file(text, f"./tts_files/{title}.mp3")
        VOICE_ENGINE.runAndWait()
        
        
def generate_videos(posts):
    for i in posts:
        title = i["data"]["title"].replace("/", "slash").replace("\\", "slash")
        text = f"r slash {i['data']['subreddit']}. {i['data']['selftext']}"
        words = text.split(" ")
        num_characters = sum([len(i) for i in words])

        voice_clip = AudioFileClip(f"./tts_files/{title}.mp3")
        background_clip = VideoFileClip(f"./background_videos/mc_parkour.mp4")
                
        subclips = []
        time_index = 0
        for j in range(len(words)):
            clip_length = 1.2*voice_clip.duration*len(words[j])/num_characters
            video_subclip = background_clip.subclip(time_index, time_index+clip_length)
            text_subclip = TextClip(words[j], align="center", size=(500, 500))
            
            subclip = CompositeVideoClip([video_subclip, text_subclip])
            subclip = subclip.set_duration(clip_length)
            subclips.append(subclip)
            
            time_index += clip_length
                
        final_video = concatenate_videoclips(subclips).set_audio(voice_clip)
        final_video.write_videofile(f"./videos/{title}.mp4")
        break
      
      
posts = get_posts()
get_tts(posts)
generate_videos(posts)